# Run Inference on StarCoder2-7B

## Install and Load Dependencies

In [1]:
!pip install accelerate

In [2]:
import torch
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

2024-06-11 14:18:52.218481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 14:18:52.218533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 14:18:52.223230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Import Model and Tokenizer

In [3]:
checkpoint = "bigcode/starcoder2-7b"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# for fp16 use `torch_dtype=torch.float16` instead
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
print(f"Memory footprint: {model.get_memory_footprint() / 1e6:.2f} MB")

Memory footprint: 14616.29 MB


## Create Test Prompt

In [5]:
prompt = """
Write a script in Python that implements a circular queue and creates one with user input. Give an example of how the script works and document the code with appropriate comments and docstrings wherever necessary. 
Generate a response only for the prompt that is given and nothing else. 
"""

## Run Inference with `model.generate()`

In [6]:
%%time
inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Write a script in Python that implements a circular queue and creates one with user input. Give an example of how the script works and document the code with appropriate comments and docstrings wherever necessary. 
Generate a response only for the prompt that is given and nothing else. 

## Solution

```python
class CircularQueue:
    def __init__(self, size):
        self.size = size
        self.queue = [None] * size
        self.front = 0
        self.rear = 0
        self.count = 0

    def enqueue(self, data):
        if self.isFull():
            print("Queue is full")
            return
        self.queue[self.rear] = data
        self.rear = (self.rear + 1) % self.size
        self.count += 1

    def dequeue(self):
        if self.isEmpty():
            print("Queue is empty")
            return
        data = self.queue[self.front]
        self.front = (self.front + 1) % self.size
        self.count -= 1
        return data

    def isFull(self):
        return self.count ==

## Run Inference with Pipeline

In [7]:
def generateOutputWithModelPipeline(model, tokenizer, prompt, temperature=0.7, max_new_tokens=256) : 
    textGenerator = pipeline(
        task="text-generation", 
        model=model, 
        tokenizer=tokenizer,
        pad_token_id=tokenizer.eos_token_id
    )
    
    start = datetime.now()
    
    output = textGenerator(
        prompt,
        do_sample=True,
        max_new_tokens=max_new_tokens, 
        temperature=temperature, 
        top_k=50, 
        top_p=0.9,
        num_return_sequences=1
    )
    
    stop = datetime.now()
    
    totalTimeToPrompt = stop - start
    print(f"Execution Time : {totalTimeToPrompt}")
    
    return output

In [8]:
output = generateOutputWithModelPipeline(model, tokenizer, prompt)
print(output[0]['generated_text'])

Execution Time : 0:00:16.299912

Write a script in Python that implements a circular queue and creates one with user input. Give an example of how the script works and document the code with appropriate comments and docstrings wherever necessary. 
Generate a response only for the prompt that is given and nothing else. 

**Note:** Do not use the built-in queue module.

**Hint:**
- Use a list to store the queue elements. 
- Create a function that returns the length of the queue.
- Create a function that checks if the queue is empty.
- Create a function that adds an element to the queue.
- Create a function that removes an element from the queue.
- Create a function that prints all the elements in the queue.

**Sample input/output:**

```
Enter the size of the queue: 10
Enter the element to be added in the queue: 1
Enter the element to be added in the queue: 2
Enter the element to be added in the queue: 3
Enter the element to be added in the queue: 4
Enter the element to be added in the q

## Test Pipeline with Custom Input

In [9]:
language = "Python"
code = """
def test_euler_method(self):
    def dydx(x, y):
        return x + y

    x0, y0 = 0, 1
    x_end = 1
    h = 0.01
    x, y = differential_equations.euler_method(dydx, x0, y0, x_end, h)
    expected = np.exp(x_end) - x_end - 1  # Analytical solution for the differential equation dy/dx = x + y
    self.assertAlmostEqual(y[-1], expected, places=2)
"""

In [10]:
template = f"""
Given a script file in {language}, generate its documentation for each function. For each function in the script, document its name, arguments, return values, and a brief explanation of its logic. Ensure that code within comments is not parsed and documented. Generate nothing else than what is asked.

Strictly use the following format for each function:

## Function Name: `function_name`

### Arguments
* `arg1` (type): Description of argument 1.
* `arg2` (type): Description of argument 2.
* ...

### Return Values
* `return_value1` (type): Description of return value 1.
* `return_value2` (type): Description of return value 2.
* ...

### Explanation of Function Logic:
1. Brief explanation of the function logic step by step.
2. ...
3. ...

-----------------------------------------------------------------------------

Here is the function I want you to generate documentation of:

{code}
"""

In [11]:
template

'\nGiven a script file in Python, generate its documentation for each function. For each function in the script, document its name, arguments, return values, and a brief explanation of its logic. Ensure that code within comments is not parsed and documented. Generate nothing else than what is asked.\n\nStrictly use the following format for each function:\n\n## Function Name: `function_name`\n\n### Arguments\n* `arg1` (type): Description of argument 1.\n* `arg2` (type): Description of argument 2.\n* ...\n\n### Return Values\n* `return_value1` (type): Description of return value 1.\n* `return_value2` (type): Description of return value 2.\n* ...\n\n### Explanation of Function Logic:\n1. Brief explanation of the function logic step by step.\n2. ...\n3. ...\n\n-----------------------------------------------------------------------------\n\nHere is the function I want you to generate documentation of:\n\n\ndef test_euler_method(self):\n    def dydx(x, y):\n        return x + y\n\n    x0, y0

In [12]:
output = generateOutputWithModelPipeline(model, tokenizer, template)
generatedText = output[0]['generated_text']
response = generatedText[len(template):].strip()

Execution Time : 0:00:16.395603


In [13]:
print(generatedText)


Given a script file in Python, generate its documentation for each function. For each function in the script, document its name, arguments, return values, and a brief explanation of its logic. Ensure that code within comments is not parsed and documented. Generate nothing else than what is asked.

Strictly use the following format for each function:

## Function Name: `function_name`

### Arguments
* `arg1` (type): Description of argument 1.
* `arg2` (type): Description of argument 2.
* ...

### Return Values
* `return_value1` (type): Description of return value 1.
* `return_value2` (type): Description of return value 2.
* ...

### Explanation of Function Logic:
1. Brief explanation of the function logic step by step.
2. ...
3. ...

-----------------------------------------------------------------------------

Here is the function I want you to generate documentation of:


def test_euler_method(self):
    def dydx(x, y):
        return x + y

    x0, y0 = 0, 1
    x_end = 1
    h = 0.

In [14]:
print(response)

"""

# Python code to read the file.
# open() function opens a file in read only mode.
file1 = open('differential_equations.py', 'r')
print("Name of the file: ", file1.name)

# read() function reads the file.
file_content = file1.read()
print(file_content)

# close() function closes the file.
file1.close()


"""

Name of the file:  differential_equations.py

# Differential Equations

import numpy as np

def euler_method(dydx, x0, y0, x_end, h):
    """
    Parameters
    ----------
    dydx: function
        A function that returns dy/dx for a given x and y.
    x0: float
        The initial value of x.
    y0: float
        The initial value of y.
    x_end: float
        The value of x to stop the integration at.
    h: float
        The integration step.

    Returns
    -------
    x: numpy.ndarray
        An array of x values.
    y: numpy.ndarray
        An array


## Remarks

I'm sure StarCoder2 is great in it's own regard as a code completion agent, however, for my use case, it simply won't cut it.